...continue microoptimization 
V8 => 0.01367 

The following kernel here shows how the excellent kernel [here](https://www.kaggle.com/scirpus/hybrid-gp-and-nn) may benefit from adding some regularisation. Overfitting was suspected because the validation scores on the neural network were considerably below the public leaderboard score. [scirpus](https://www.kaggle.com/scirpus) produces a lot of high quality kernels for Kaggle so please be sure to upvote the kernel listed.

In [1]:
import numpy as np
import pandas as pd
import sklearn.metrics as mtr
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from tqdm import tqdm_notebook

import gc
gc.enable()

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 150)

In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, PReLU, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import Callback, EarlyStopping
from keras.optimizers import Adam
from keras import backend as K
from keras import regularizers
import tensorflow as tf

Using TensorFlow backend.


In [3]:
train = pd.read_csv('../data/train.csv', usecols=['GameId', 'PlayId', 'Team', 'X', 'Y', 'S', 'A', 'Dis',
                                                                               'Orientation', 'Dir', 'NflId', 'DisplayName', 'YardLine',
                                                                               'Quarter', 'GameClock', 'PossessionTeam', 'Down', 'Distance',
                                                                               'FieldPosition', 'NflIdRusher', 'OffenseFormation', 
                                                                               'OffensePersonnel', 'DefendersInTheBox', 'DefensePersonnel', 
                                                                               'PlayDirection', 'TimeHandoff', 'TimeSnap', 'Yards',
                                                                              'WindSpeed', 'Temperature', 'Humidity'])

/Users/rodrigolima82/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#train = train[:2200]
outcomes = train[['GameId','PlayId','Yards']].drop_duplicates()

## Functions for anchoring offense moving left from {0,0}

In [5]:
def create_features(df, deploy=False):
    def new_X(x_coordinate, play_direction):
        if play_direction == 'left':
            return 120.0 - x_coordinate
        else:
            return x_coordinate
        
    # Adicionado por rodrigo
    def new_Y(y_coordinate, play_direction):
        if play_direction == 'left':
            return (160 / 3) - y_coordinate
        else:
            return y_coordinate
        
    def new_line(rush_team, field_position, yardline):
        if rush_team == field_position:
            # offense starting at X = 0 plus the 10 yard endzone plus the line of scrimmage
            return 10.0 + yardline
        else:
            # half the field plus the yards between midfield and the line of scrimmage
            return 60.0 + (50 - yardline)

    def new_orientation(angle, play_direction):
        if play_direction == 'left':
            new_angle = 360.0 - angle
            if new_angle == 360.0:
                new_angle = 0.0
            return new_angle
        else:
            return angle

    def euclidean_distance(x1,y1,x2,y2):
        x_diff = (x1-x2)**2
        y_diff = (y1-y2)**2

        return np.sqrt(x_diff + y_diff)

    def back_direction(orientation):
        if orientation > 180.0:
            return 1
        else:
            return 0

    def update_yardline(df):
        new_yardline = df[df['NflId'] == df['NflIdRusher']]
        new_yardline['YardLine'] = new_yardline[['PossessionTeam','FieldPosition','YardLine']].apply(lambda x: new_line(x[0],x[1],x[2]), axis=1)
        new_yardline = new_yardline[['GameId','PlayId','YardLine']]

        return new_yardline

    def update_orientation(df, yardline):
        df['X'] = df[['X','PlayDirection']].apply(lambda x: new_X(x[0],x[1]), axis=1)
        df['Y'] = df[['Y','PlayDirection']].apply(lambda x: new_Y(x[0],x[1]), axis=1)
        df['Orientation'] = df[['Orientation','PlayDirection']].apply(lambda x: new_orientation(x[0],x[1]), axis=1)
        df['Dir'] = df[['Dir','PlayDirection']].apply(lambda x: new_orientation(x[0],x[1]), axis=1)

        df = df.drop('YardLine', axis=1)
        df = pd.merge(df, yardline, on=['GameId','PlayId'], how='inner')

        return df
    
    def projection_features(df):
        rad = 2 * np.pi * (90 - df[['Orientation']]) / 360
        v_0 = df['S'].values * np.cos(rad).values.reshape(-1)
        v_1 = np.sin(rad).values.reshape(-1)

        a_0 = df['A'].values * np.cos(rad).values.reshape(-1)
        a_1 = np.sin(rad)

        df['v_0'] = v_0
        df['v_1'] = v_1
        df['a_0'] = a_0
        df['a_1'] = a_1

        df = df.groupby(['GameId','PlayId'])\
                         .agg({'v_0':'mean',
                               'v_1':'mean',
                               'a_0':'mean',
                               'a_1':'mean',})\
                         .reset_index()
        df.columns = ['GameId','PlayId','v_0_mean','v_1_mean','a_0_mean','a_1_mean']

        return df

    def back_features(df):
        carriers = df[df['NflId'] == df['NflIdRusher']][['GameId','PlayId','NflIdRusher','X','Y','Orientation','Dir','YardLine']]
        carriers['back_from_scrimmage'] = carriers['YardLine'] - carriers['X']
        carriers['back_oriented_down_field'] = carriers['Orientation'].apply(lambda x: back_direction(x))
        carriers['back_moving_down_field'] = carriers['Dir'].apply(lambda x: back_direction(x))
        carriers = carriers.rename(columns={'X':'back_X',
                                            'Y':'back_Y'})
        carriers = carriers[['GameId','PlayId','NflIdRusher','back_X','back_Y','back_from_scrimmage','back_oriented_down_field','back_moving_down_field']]

        return carriers

    def features_relative_to_back(df, carriers):
        player_distance = df[['GameId','PlayId','NflId','X','Y']]
        player_distance = pd.merge(player_distance, carriers, on=['GameId','PlayId'], how='inner')
        player_distance = player_distance[player_distance['NflId'] != player_distance['NflIdRusher']]
        player_distance['dist_to_back'] = player_distance[['X','Y','back_X','back_Y']].apply(lambda x: euclidean_distance(x[0],x[1],x[2],x[3]), axis=1)

        player_distance = player_distance.groupby(['GameId','PlayId','back_from_scrimmage','back_oriented_down_field','back_moving_down_field'])\
                                         .agg({'dist_to_back':['min','max','mean','std']})\
                                         .reset_index()
        player_distance.columns = ['GameId','PlayId','back_from_scrimmage','back_oriented_down_field','back_moving_down_field',
                                   'min_dist','max_dist','mean_dist','std_dist']

        return player_distance

    def defense_features(df):
        rusher = df[df['NflId'] == df['NflIdRusher']][['GameId','PlayId','Team','X','Y']]
        rusher.columns = ['GameId','PlayId','RusherTeam','RusherX','RusherY']

        defense = pd.merge(df,rusher,on=['GameId','PlayId'],how='inner')
        defense = defense[defense['Team'] != defense['RusherTeam']][['GameId','PlayId','X','Y','RusherX','RusherY']]
        defense['def_dist_to_back'] = defense[['X','Y','RusherX','RusherY']].apply(lambda x: euclidean_distance(x[0],x[1],x[2],x[3]), axis=1)

        defense = defense.groupby(['GameId','PlayId'])\
                         .agg({'def_dist_to_back':['min','max','mean','std']})\
                         .reset_index()
        defense.columns = ['GameId','PlayId','def_min_dist','def_max_dist','def_mean_dist','def_std_dist']


        return defense

    def static_features(df):
        static_features = df[df['NflId'] == df['NflIdRusher']][['GameId','PlayId','X','Y','S','A','Dis','Orientation','Dir',
                                                            'YardLine','Quarter','Down','Distance','DefendersInTheBox']].drop_duplicates()
        static_features['DefendersInTheBox'] = static_features['DefendersInTheBox'].fillna(np.mean(static_features['DefendersInTheBox']))

        return static_features


    def combine_features(relative_to_back, defense, static, speed_frame, deploy=deploy):
        df = pd.merge(relative_to_back,defense,on=['GameId','PlayId'],how='inner')
        df = pd.merge(df,static,on=['GameId','PlayId'],how='inner')
        df = pd.merge(df, speed_frame,on=['GameId','PlayId'],how='inner')
        
        if not deploy:
            df = pd.merge(df, outcomes, on=['GameId','PlayId'], how='inner')

        return df
    
    yardline = update_yardline(df)
    df = update_orientation(df, yardline)
    back_feats = back_features(df)
    rel_back = features_relative_to_back(df, back_feats)
    def_feats = defense_features(df)
    static_feats = static_features(df)
    speed_frame = projection_features(df)
    
    basetable = combine_features(rel_back, def_feats, static_feats,  speed_frame, deploy=deploy)
    #basetable[['a_0','a_1','v_0','v_1']] = basetable[['a_0','a_1','v_0','v_1']].fillna(0)
    #basetable['Temperature'] = basetable['Temperature'].fillna(60.436442)
    #basetable['Humidity'] =     basetable['Humidity'].fillna(59.0)
    return basetable

In [6]:
%time train_basetable = create_features(train, False)

/Users/rodrigolima82/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


CPU times: user 1min 40s, sys: 2.39 s, total: 1min 42s
Wall time: 1min 37s


In [7]:
train_basetable.shape

(23171, 30)

In [8]:
train_basetable.head()

,GameId,PlayId,back_from_scrimmage,back_oriented_down_field,back_moving_down_field,min_dist,max_dist,mean_dist,std_dist,def_min_dist,def_max_dist,def_mean_dist,def_std_dist,X,Y,S,A,Dis,Orientation,Dir,YardLine,Quarter,Down,Distance,DefendersInTheBox,v_0_mean,v_1_mean,a_0_mean,a_1_mean,Yards
0,2017090700,20170907000118,3.75,1,0,1.449724,22.415872,8.046559,4.873845,4.593310,22.415872,9.752491,5.327299,41.25,22.803333,3.63,3.35,0.38,198.02,114.26,45.0,1,3,2,6.0,-0.468750,-0.035180,-0.105794,-0.035180,8
1,2017090700,20170907000139,4.07,0,0,0.792023,23.025872,8.614623,5.598683,4.287773,23.025872,10.297028,5.833217,48.93,26.173333,3.06,2.41,0.34,149.30,47.80,53.0,1,1,10,6.0,0.097014,-0.019960,0.047904,-0.019960,3
2,2017090700,20170907000189,3.66,1,0,1.646390,20.726285,8.482583,4.642121,4.221670,20.726285,9.903689,5.073290,71.34,34.223333,5.77,2.42,0.60,219.18,138.04,75.0,1,1,10,7.0,-1.300817,0.147571,-0.680036,0.147571,5
3,2017090700,20170907000345,3.53,0,0,0.918096,9.791231,5.549379,1.983128,4.528002,9.791231,6.309354,1.834174,104.47,27.973333,4.45,3.20,0.46,173.78,84.56,108.0,1,2,2,9.0,0.453548,0.109604,0.114001,0.109604,2
4,2017090700,20170907000395,5.01,0,0,0.502892,21.214806,9.168819,5.611232,4.288088,21.214806,11.056456,5.900009,29.99,27.120000,3.90,2.53,0.44,34.27,157.92,35.0,1,1,10,7.0,0.624800,0.004144,0.464362,0.004144,7


In [9]:
# Fazendo uma limpeza na memoria
gc.collect()

105

# Let's split our data into train/val

In [10]:
X = train_basetable.copy()
yards = X.Yards

y = np.zeros((yards.shape[0], 199))
for idx, target in enumerate(list(yards)):
    y[idx][99 + target] = 1

X.drop(['GameId','PlayId','Yards'], axis=1, inplace=True)
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [11]:
X.shape, y.shape

((23171, 27), (23171, 199))

# Let's build NN

Below class Metric based entirely on: https://www.kaggle.com/kingychiu/keras-nn-starter-crps-early-stopping
<br></br>
Below early stopping entirely based on: https://www.kaggle.com/c/nfl-big-data-bowl-2020/discussion/112868#latest-656533

In [12]:
class Metric(Callback):
    def __init__(self, model, callbacks, data):
        super().__init__()
        self.model = model
        self.callbacks = callbacks
        self.data = data

    def on_train_begin(self, logs=None):
        for callback in self.callbacks:
            callback.on_train_begin(logs)

    def on_train_end(self, logs=None):
        for callback in self.callbacks:
            callback.on_train_end(logs)

    def on_epoch_end(self, batch, logs=None):
        X_train, y_train = self.data[0][0], self.data[0][1]
        y_pred = self.model.predict(X_train)
        y_true = np.clip(np.cumsum(y_train, axis=1), 0, 1)
        y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
        tr_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * X_train.shape[0])
        tr_s = np.round(tr_s, 6)
        logs['tr_CRPS'] = tr_s

        X_valid, y_valid = self.data[1][0], self.data[1][1]

        y_pred = self.model.predict(X_valid)
        y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
        y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
        val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * X_valid.shape[0])
        val_s = np.round(val_s, 6)
        logs['val_CRPS'] = val_s
        print('tr CRPS', tr_s, 'val CRPS', val_s)

        for callback in self.callbacks:
            callback.on_epoch_end(batch, logs)

In [13]:
keras.backend.clear_session()
def crps(y_true, y_pred):
    loss = K.mean((K.cumsum(y_pred, axis = 1) - y_true)**2)
    return loss

def get_model(batch_size, epochs, x_tr, y_tr, x_vl, y_vl):
    model = Sequential()
    model.add(Dense(512, input_dim=X.shape[1], activation=None))
    model.add(PReLU())
    model.add(Dropout(0.5))
    model.add(Dense(256, activation=None))
    model.add(PReLU())
    model.add(Dropout(0.2))
    model.add(Dense(199, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(learning_rate=0.006, decay = 1e-5),
                  metrics=[])

    es = EarlyStopping(monitor='val_CRPS', 
                       mode='min',
                       restore_best_weights=True, 
                       verbose=1, 
                       patience=15,
                       min_delta=1e-4)
    es.set_model(model)
    metric = Metric(model, [es], [(x_tr, y_tr), (x_vl, y_vl)])
    model.fit(x_tr, y_tr, callbacks=[metric], epochs=epochs, batch_size=batch_size, validation_data=[x_vl, y_vl])
    loss = model.history.history["val_CRPS"][-1]

    return model, loss

In [14]:
losses = []
models = []
#for k in range(2):
kfold = KFold(100, shuffle = True)
for k_fold, (tr_idx, vl_idx) in enumerate(kfold.split(X, y)):
    print("-----------")

    x_tr, y_tr = X[tr_idx], y[tr_idx]
    x_vl, y_vl = X[vl_idx], y[vl_idx]    

    model, loss = get_model(1024, 250, x_tr, y_tr, x_vl, y_vl)    
    if (loss < 0.013):
        print('Append KFold:',k_fold,'| Loss:', loss)
        models.append(model)
        losses.append(loss)
        if (len(losses) > 10):
            break
    else:
        print('Ignore KFold:',k_fold, '| Loss:', loss)        
print("-------")
print('Loss Mean:', np.mean(losses), ' | List:', losses)

-----------
Train on 22939 samples, validate on 232 samples
Epoch 1/250
22939/22939 [==============================] - 1s 41us/step - loss: 3.2249 - val_loss: 2.8568
tr CRPS 0.013521 val CRPS 0.014203
Epoch 2/250
22939/22939 [==============================] - 1s 32us/step - loss: 2.8380 - val_loss: 2.7750
tr CRPS 0.013124 val CRPS 0.013582
Epoch 3/250
22939/22939 [==============================] - 1s 29us/step - loss: 2.7894 - val_loss: 2.7596
tr CRPS 0.013039 val CRPS 0.013618
Epoch 4/250
22939/22939 [==============================] - 1s 30us/step - loss: 2.7696 - val_loss: 2.7510
tr CRPS 0.012958 val CRPS 0.013613
Epoch 5/250
22939/22939 [==============================] - 1s 29us/step - loss: 2.7584 - val_loss: 2.7175
tr CRPS 0.012925 val CRPS 0.013562
Epoch 6/250
22939/22939 [==============================] - 1s 31us/step - loss: 2.7404 - val_loss: 2.7277
tr CRPS 0.012876 val CRPS 0.013473
Epoch 7/250
22939/22939 [==============================] - 1s 33us/step - loss: 2.7402 - val_l

22939/22939 [==============================] - 1s 36us/step - loss: 2.5601 - val_loss: 2.9131
tr CRPS 0.011768 val CRPS 0.015407
Epoch 34/250
22939/22939 [==============================] - 1s 28us/step - loss: 2.5583 - val_loss: 2.9009
tr CRPS 0.011705 val CRPS 0.015394
Epoch 35/250
22939/22939 [==============================] - 1s 29us/step - loss: 2.5542 - val_loss: 2.8878
tr CRPS 0.011583 val CRPS 0.015277
Epoch 36/250
22939/22939 [==============================] - 1s 37us/step - loss: 2.5496 - val_loss: 2.9041
tr CRPS 0.011483 val CRPS 0.015301
Epoch 37/250
22939/22939 [==============================] - 1s 35us/step - loss: 2.5388 - val_loss: 2.9074
tr CRPS 0.011442 val CRPS 0.015258
Epoch 38/250
22939/22939 [==============================] - 1s 31us/step - loss: 2.5335 - val_loss: 2.8964
tr CRPS 0.011438 val CRPS 0.015281
Epoch 39/250
22939/22939 [==============================] - 1s 30us/step - loss: 2.5285 - val_loss: 2.9293
tr CRPS 0.011413 val CRPS 0.015406
Epoch 40/250
22939/

22939/22939 [==============================] - 1s 24us/step - loss: 2.7596 - val_loss: 2.8346
tr CRPS 0.01288 val CRPS 0.015245
Epoch 6/250
22939/22939 [==============================] - 1s 23us/step - loss: 2.7429 - val_loss: 2.8180
tr CRPS 0.012835 val CRPS 0.015254
Epoch 7/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.7327 - val_loss: 2.8208
tr CRPS 0.012787 val CRPS 0.015142
Epoch 8/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.7217 - val_loss: 2.8098
tr CRPS 0.01273 val CRPS 0.015188
Epoch 9/250
22939/22939 [==============================] - 1s 29us/step - loss: 2.7163 - val_loss: 2.8045
tr CRPS 0.012711 val CRPS 0.015194
Epoch 10/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.7046 - val_loss: 2.8074
tr CRPS 0.012659 val CRPS 0.015125
Epoch 11/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6968 - val_loss: 2.8159
tr CRPS 0.012662 val CRPS 0.015099
Epoch 12/250
22939/22939 

tr CRPS 0.012441 val CRPS 0.010787
Epoch 18/250
22939/22939 [==============================] - 1s 23us/step - loss: 2.6491 - val_loss: 2.5873
tr CRPS 0.0124 val CRPS 0.010702
Epoch 19/250
22939/22939 [==============================] - 1s 23us/step - loss: 2.6387 - val_loss: 2.6061
tr CRPS 0.012359 val CRPS 0.010653
Epoch 20/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6346 - val_loss: 2.6242
tr CRPS 0.012377 val CRPS 0.010713
Epoch 21/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6312 - val_loss: 2.6240
tr CRPS 0.012299 val CRPS 0.010785
Epoch 22/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6192 - val_loss: 2.5934
tr CRPS 0.012238 val CRPS 0.01068
Epoch 23/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6165 - val_loss: 2.6099
tr CRPS 0.012208 val CRPS 0.010658
Epoch 24/250
22939/22939 [==============================] - 1s 26us/step - loss: 2.6086 - val_loss: 2.6180
tr CRPS 0

22939/22939 [==============================] - 1s 23us/step - loss: 2.6277 - val_loss: 2.7991
tr CRPS 0.012269 val CRPS 0.014405
Epoch 25/250
22939/22939 [==============================] - 1s 23us/step - loss: 2.6163 - val_loss: 2.8208
tr CRPS 0.012164 val CRPS 0.014515
Epoch 26/250
22939/22939 [==============================] - 1s 23us/step - loss: 2.6095 - val_loss: 2.8269
tr CRPS 0.012079 val CRPS 0.014633
Epoch 27/250
22939/22939 [==============================] - 1s 23us/step - loss: 2.6004 - val_loss: 2.8202
tr CRPS 0.012018 val CRPS 0.014408
Epoch 28/250
22939/22939 [==============================] - 1s 23us/step - loss: 2.6018 - val_loss: 2.8258
tr CRPS 0.012013 val CRPS 0.01438
Epoch 29/250
22939/22939 [==============================] - 1s 23us/step - loss: 2.5891 - val_loss: 2.8260
tr CRPS 0.012005 val CRPS 0.014403
Epoch 30/250
22939/22939 [==============================] - 1s 23us/step - loss: 2.5873 - val_loss: 2.8312
tr CRPS 0.011909 val CRPS 0.014513
Epoch 31/250
22939/2

...

22939/22939 [==============================] - 1s 24us/step - loss: 2.6686 - val_loss: 2.8978
tr CRPS 0.012544 val CRPS 0.013806
Epoch 16/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6559 - val_loss: 2.8831
tr CRPS 0.01245 val CRPS 0.01378
Epoch 17/250
22939/22939 [==============================] - 1s 23us/step - loss: 2.6507 - val_loss: 2.8891
tr CRPS 0.012484 val CRPS 0.013682
Epoch 18/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6475 - val_loss: 2.9283
tr CRPS 0.012383 val CRPS 0.013748
Restoring model weights from the end of the best epoch
Epoch 00018: early stopping
Ignore KFold: 11 | Loss: 0.013748
-----------
Train on 22939 samples, validate on 232 samples
Epoch 1/250
22939/22939 [==============================] - 1s 34us/step - loss: 3.2236 - val_loss: 2.8006
tr CRPS 0.013573 val CRPS 0.012217
Epoch 2/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.8459 - val_loss: 2.7064
tr CRPS 0.013186 val CR

22939/22939 [==============================] - 1s 24us/step - loss: 2.6243 - val_loss: 2.7783
tr CRPS 0.012217 val CRPS 0.013377
Epoch 23/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6199 - val_loss: 2.7794
tr CRPS 0.012196 val CRPS 0.013356
Epoch 24/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6112 - val_loss: 2.7723
tr CRPS 0.012113 val CRPS 0.01333
Epoch 25/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6062 - val_loss: 2.7731
tr CRPS 0.01207 val CRPS 0.013305
Epoch 26/250
22939/22939 [==============================] - 1s 26us/step - loss: 2.6003 - val_loss: 2.8038
tr CRPS 0.012037 val CRPS 0.013552
Epoch 27/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.5938 - val_loss: 2.7842
tr CRPS 0.011938 val CRPS 0.013269
Restoring model weights from the end of the best epoch
Epoch 00027: early stopping
Ignore KFold: 13 | Loss: 0.013269
-----------
Train on 22939 samples, validate o

22939/22939 [==============================] - 1s 24us/step - loss: 2.6451 - val_loss: 2.7437
tr CRPS 0.012392 val CRPS 0.012241
Epoch 19/250
22939/22939 [==============================] - 1s 23us/step - loss: 2.6465 - val_loss: 2.7836
tr CRPS 0.012381 val CRPS 0.012373
Epoch 20/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6392 - val_loss: 2.7831
tr CRPS 0.012345 val CRPS 0.012528
Epoch 21/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6282 - val_loss: 2.8009
tr CRPS 0.012265 val CRPS 0.0125
Epoch 22/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6272 - val_loss: 2.7892
tr CRPS 0.012268 val CRPS 0.012425
Epoch 23/250
22939/22939 [==============================] - 1s 23us/step - loss: 2.6251 - val_loss: 2.7892
tr CRPS 0.01227 val CRPS 0.0125
Epoch 24/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6094 - val_loss: 2.7892
tr CRPS 0.01215 val CRPS 0.012386
Epoch 25/250
22939/22939 

22939/22939 [==============================] - 1s 24us/step - loss: 2.6695 - val_loss: 2.7889
tr CRPS 0.01257 val CRPS 0.014926
Epoch 17/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6568 - val_loss: 2.7783
tr CRPS 0.012443 val CRPS 0.014902
Epoch 18/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6461 - val_loss: 2.8056
tr CRPS 0.012373 val CRPS 0.014844
Epoch 19/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6362 - val_loss: 2.8187
tr CRPS 0.012313 val CRPS 0.014832
Epoch 20/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6304 - val_loss: 2.8190
tr CRPS 0.012258 val CRPS 0.014938
Epoch 21/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6249 - val_loss: 2.8040
tr CRPS 0.012192 val CRPS 0.014888
Epoch 22/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6211 - val_loss: 2.7910
tr CRPS 0.012153 val CRPS 0.01488
Epoch 23/250
22939/22

22939/22939 [==============================] - 1s 24us/step - loss: 2.6620 - val_loss: 2.7599
tr CRPS 0.012583 val CRPS 0.012689
Epoch 17/250
22939/22939 [==============================] - 1s 23us/step - loss: 2.6500 - val_loss: 2.7586
tr CRPS 0.012444 val CRPS 0.012674
Epoch 18/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6495 - val_loss: 2.7982
tr CRPS 0.012424 val CRPS 0.012716
Epoch 19/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6510 - val_loss: 2.7681
tr CRPS 0.012379 val CRPS 0.012647
Epoch 20/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6330 - val_loss: 2.7665
tr CRPS 0.012335 val CRPS 0.012684
Epoch 21/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6305 - val_loss: 2.8033
tr CRPS 0.012266 val CRPS 0.012728
Epoch 22/250
22939/22939 [==============================] - 1s 24us/step - loss: 2.6249 - val_loss: 2.7619
tr CRPS 0.012233 val CRPS 0.012628
Epoch 23/250
22939/

In [15]:
print(losses)
print(np.mean(losses))

[0.010974, 0.011286, 0.012174, 0.011405, 0.010976, 0.01092, 0.011209, 0.012434, 0.011769, 0.012773, 0.012575]
0.011681363636363636


In [16]:
class GP:
    def __init__(self):
        self.classes = 20
        self.class_names = [ 'class_0',
                             'class_1',
                             'class_2',
                             'class_3',
                             'class_4',
                             'class_5',
                             'class_6',
                             'class_7',
                             'class_8',
                             'class_9',
                             'class_10',
                             'class_11',
                             'class_12',
                             'class_13',
                            'class_14',
                            'class_15',
                            'class_16',
                            'class_17',
                            'class_18',
                            'class_19',
                           ]


    def GrabPredictions(self, data):
        oof_preds = np.zeros((len(data), len(self.class_names)))
        oof_preds[:,0] = self.GP_class_0(data)
        oof_preds[:,1] = self.GP_class_1(data)
        oof_preds[:,2] = self.GP_class_2(data)
        oof_preds[:,3] = self.GP_class_3(data)
        oof_preds[:,4] = self.GP_class_4(data)
        oof_preds[:,5] = self.GP_class_5(data)
        oof_preds[:,6] = self.GP_class_6(data)
        oof_preds[:,7] = self.GP_class_7(data)
        oof_preds[:,8] = self.GP_class_8(data)
        oof_preds[:,9] = self.GP_class_9(data)
        oof_preds[:,10] = self.GP_class_10(data)
        oof_preds[:,11] = self.GP_class_11(data)
        oof_preds[:,12] = self.GP_class_12(data)
        oof_preds[:,13] = self.GP_class_13(data)
        oof_preds[:,14] = self.GP_class_14(data)
        oof_preds[:,15] = self.GP_class_15(data)
        oof_preds[:,16] = self.GP_class_16(data)
        oof_preds[:,17] = self.GP_class_17(data)
        oof_preds[:,18] = self.GP_class_18(data)
        oof_preds[:,19] = self.GP_class_19(data)
        oof_df = pd.DataFrame(np.exp(oof_preds), columns=self.class_names)
        oof_df =oof_df.div(oof_df.sum(axis=1), axis=0)
        
        return oof_df.values


    def GP_class_0(self,data):
        return(0.250000*np.tanh(((((((data[:,0]) - (((data[:,0]) + (data[:,0]))))) + (data[:,0]))) + (((((data[:,0]) * 2.0)) * 2.0)))) +
                0.250000*np.tanh(((((((((((((data[:,2]) - (data[:,7]))) * 2.0)) + (data[:,7]))) * 2.0)) + (data[:,0]))) + (((((data[:,2]) + (data[:,0]))) - (data[:,7]))))) +
                0.250000*np.tanh(((((((((((((((((((data[:,3]) - (data[:,7]))) - (((data[:,0]) / 2.0)))) + (data[:,0]))) - (data[:,7]))) + (data[:,2]))) + (((data[:,7]) + (data[:,2]))))) + (data[:,7]))) - (data[:,7]))) + (data[:,0]))) +
                0.250000*np.tanh(((((((((data[:,0]) - (data[:,7]))) - (data[:,14]))) + (((data[:,17]) - (data[:,14]))))) + (((((((data[:,0]) - ((1.0)))) - (data[:,0]))) + (data[:,20]))))) +
                0.250000*np.tanh(((((data[:,18]) + (((((((data[:,0]) - (data[:,7]))) + (data[:,2]))) + (data[:,0]))))) - (((data[:,7]) + (data[:,2]))))) +
                0.250000*np.tanh((((((((data[:,8]) + ((((((((data[:,13]) + (data[:,3]))) + (data[:,0]))/2.0)) + ((((((((data[:,0]) + (((((data[:,0]) - (data[:,0]))) / 2.0)))) + (data[:,17]))/2.0)) + (data[:,11]))))))/2.0)) + (data[:,0]))) - (data[:,13]))) +
                0.250000*np.tanh(((((data[:,0]) * 2.0)) + (((((((((data[:,2]) + (data[:,2]))) * 2.0)) - (((data[:,14]) - (((np.tanh((data[:,2]))) * 2.0)))))) - (data[:,7]))))) +
                0.250000*np.tanh(((((((((((((data[:,17]) - (((data[:,17]) * (((((data[:,17]) * (data[:,17]))) * 2.0)))))) + (((((((data[:,17]) * 2.0)) * 2.0)) * 2.0)))/2.0)) / 2.0)) + (((data[:,17]) - ((7.76236486434936523)))))/2.0)) + (((((((data[:,17]) * (data[:,17]))) * 2.0)) * 2.0)))) +
                0.250000*np.tanh(((((((((((data[:,17]) + (data[:,17]))/2.0)) + ((((data[:,17]) + ((((data[:,17]) + (data[:,17]))/2.0)))/2.0)))/2.0)) * (data[:,17]))) - (data[:,13]))) +
                0.250000*np.tanh((((((data[:,19]) + (((((((((data[:,19]) * 2.0)) + (data[:,19]))) - ((1.07645297050476074)))) + (((data[:,22]) + (data[:,22]))))))/2.0)) + (((data[:,22]) + (data[:,4]))))))
    
    def GP_class_1(self,data):
        return(0.250000*np.tanh(((((((((((((data[:,3]) + (((np.tanh((data[:,2]))) - (data[:,14]))))) - ((10.0)))) - ((4.88989353179931641)))) - ((13.92175483703613281)))) / 2.0)) - (data[:,6]))) +
                0.250000*np.tanh(data[:,3]) +
                0.250000*np.tanh(((data[:,2]) + (((((np.tanh((((data[:,14]) * 2.0)))) - (((((((data[:,7]) * 2.0)) - (((((data[:,0]) - (data[:,14]))) - (data[:,7]))))) - (data[:,0]))))) + (data[:,0]))))) +
                0.250000*np.tanh(((((((data[:,0]) - ((-1.0*((data[:,14])))))) + ((((((((((data[:,0]) - (data[:,7]))) - ((((data[:,14]) + (data[:,14]))/2.0)))) + (((data[:,2]) - (((data[:,0]) * (data[:,14]))))))/2.0)) - (data[:,14]))))) - (data[:,7]))) +
                0.250000*np.tanh(((((data[:,0]) / 2.0)) + (data[:,0]))) +
                0.250000*np.tanh(((((data[:,19]) - (data[:,19]))) / 2.0)) +
                0.250000*np.tanh(((((np.tanh((((np.tanh((data[:,5]))) - ((((1.69341480731964111)) + ((((((np.tanh((data[:,14]))) - (data[:,14]))) + (data[:,14]))/2.0)))))))) - (data[:,2]))) - (((data[:,14]) * 2.0)))) +
                0.250000*np.tanh((((((data[:,22]) + (((data[:,0]) + ((((((((data[:,22]) / 2.0)) + ((-1.0*((data[:,13])))))/2.0)) + (data[:,19]))))))/2.0)) - (data[:,13]))) +
                0.250000*np.tanh(((((((data[:,17]) / 2.0)) * (data[:,17]))) * (((data[:,17]) + (((data[:,17]) * (((((((((data[:,17]) + (data[:,17]))) * (data[:,17]))) * (data[:,17]))) + (((np.tanh((data[:,17]))) / 2.0)))))))))) +
                0.250000*np.tanh((((((data[:,14]) * (((data[:,13]) + (data[:,3]))))) + (((data[:,14]) * (((data[:,14]) - ((-1.0*((data[:,3])))))))))/2.0)))
    
    def GP_class_2(self,data):
        return(0.250000*np.tanh(((data[:,0]) + (np.tanh((((((((((data[:,2]) + (((data[:,2]) / 2.0)))/2.0)) + (data[:,0]))/2.0)) * (data[:,7]))))))) +
                0.250000*np.tanh((((data[:,22]) + (data[:,22]))/2.0)) +
                0.250000*np.tanh((((((data[:,14]) + ((-1.0*((((((4.73206138610839844)) + (((((((((((np.tanh((data[:,7]))) + (data[:,1]))/2.0)) - (np.tanh((((((data[:,0]) - (data[:,20]))) - (data[:,14]))))))) * ((7.0)))) + (data[:,14]))/2.0)))/2.0))))))/2.0)) - (data[:,14]))) +
                0.250000*np.tanh(((((data[:,1]) + ((4.15603733062744141)))) / 2.0)) +
                0.250000*np.tanh(((((((((((((data[:,0]) - (data[:,7]))) - (data[:,7]))) + ((((data[:,0]) + (data[:,0]))/2.0)))) * 2.0)) + (data[:,0]))) * 2.0)) +
                0.250000*np.tanh(((((data[:,11]) / 2.0)) / 2.0)) +
                0.250000*np.tanh(((((((((((((((((((((((data[:,2]) / 2.0)) + (((data[:,2]) / 2.0)))/2.0)) / 2.0)) / 2.0)) / 2.0)) + ((((-1.0*((((data[:,0]) / 2.0))))) * (((data[:,0]) / 2.0)))))/2.0)) + (data[:,0]))/2.0)) + (np.tanh((data[:,0]))))/2.0)) / 2.0)) +
                0.250000*np.tanh(((((data[:,0]) + (((data[:,0]) + (((((((data[:,14]) * (data[:,14]))) + (data[:,14]))) - (data[:,0]))))))) - (data[:,14]))) +
                0.250000*np.tanh((((((np.tanh((np.tanh(((0.0)))))) * (data[:,15]))) + (data[:,20]))/2.0)) +
                0.250000*np.tanh(((data[:,14]) * ((((data[:,13]) + ((((data[:,13]) + ((((data[:,14]) + ((((((data[:,14]) + ((((data[:,14]) + (((data[:,13]) - ((((-1.0*(((((data[:,14]) + (data[:,14]))/2.0))))) * 2.0)))))/2.0)))) + (data[:,14]))/2.0)))/2.0)))/2.0)))/2.0)))))
    
    def GP_class_3(self,data):
        return(0.250000*np.tanh(((((((((8.0)) + (((((((5.33416414260864258)) + ((9.0)))/2.0)) + ((((8.0)) * 2.0)))))) * 2.0)) + ((5.33416414260864258)))/2.0)) +
                0.250000*np.tanh((((9.48088836669921875)) + (((((10.56953334808349609)) + ((((4.48959350585937500)) + (np.tanh(((((3.0)) + ((9.0)))))))))/2.0)))) +
                0.250000*np.tanh((((((((((data[:,22]) / 2.0)) + (data[:,22]))/2.0)) * 2.0)) / 2.0)) +
                0.250000*np.tanh((10.44883441925048828)) +
                0.250000*np.tanh(((((data[:,0]) + (((data[:,0]) + ((-1.0*((data[:,9])))))))) + (((((data[:,0]) - (data[:,7]))) - (data[:,9]))))) +
                0.250000*np.tanh(((np.tanh((data[:,6]))) - (data[:,21]))) +
                0.250000*np.tanh(((data[:,0]) - (((((data[:,14]) + (((((data[:,14]) + (((data[:,14]) + (((data[:,14]) - (data[:,0]))))))) - (((((data[:,0]) - (data[:,14]))) + (data[:,14]))))))) - (((((data[:,14]) * (data[:,14]))) + (data[:,18]))))))) +
                0.250000*np.tanh(((data[:,14]) * ((((data[:,13]) + (data[:,14]))/2.0)))) +
                0.250000*np.tanh((((((data[:,8]) * (((data[:,8]) / 2.0)))) + ((((((data[:,9]) * (data[:,8]))) + (data[:,8]))/2.0)))/2.0)) +
                0.250000*np.tanh((((data[:,2]) + (((((((data[:,2]) + (data[:,20]))/2.0)) + ((((((((data[:,2]) + (data[:,8]))/2.0)) / 2.0)) - (data[:,8]))))/2.0)))/2.0)))
    
    def GP_class_4(self,data):
        return(0.250000*np.tanh((((12.98819637298583984)) / 2.0)) +
                0.250000*np.tanh(((((4.75780344009399414)) + (((((((8.66014671325683594)) + ((4.18107128143310547)))/2.0)) * ((8.97841739654541016)))))/2.0)) +
                0.250000*np.tanh((((9.0)) / 2.0)) +
                0.250000*np.tanh(((((((((((10.0)) + (np.tanh((((((6.0)) + ((((13.80149555206298828)) + ((7.0)))))/2.0)))))) + ((10.0)))) + (((((((data[:,0]) + ((8.0)))) / 2.0)) / 2.0)))/2.0)) + ((((-1.0*((((data[:,2]) * (data[:,9])))))) * 2.0)))) +
                0.250000*np.tanh(((((data[:,0]) + (((((((data[:,22]) + (data[:,22]))/2.0)) + (data[:,22]))/2.0)))) + ((((((data[:,22]) + ((((((data[:,22]) * 2.0)) + (data[:,22]))/2.0)))) + (data[:,18]))/2.0)))) +
                0.250000*np.tanh(((((((((((((np.tanh((data[:,18]))) + ((((data[:,1]) + (data[:,18]))/2.0)))/2.0)) * (data[:,18]))) * (data[:,18]))) + (((data[:,18]) * (data[:,18]))))/2.0)) - (((((((np.tanh((data[:,18]))) + (((data[:,18]) * (data[:,18]))))/2.0)) + (data[:,17]))/2.0)))) +
                0.250000*np.tanh(((np.tanh(((((((np.tanh((data[:,6]))) - (((np.tanh((data[:,6]))) - (data[:,6]))))) + (np.tanh((data[:,6]))))/2.0)))) - ((((data[:,10]) + ((((data[:,2]) + (((np.tanh((data[:,6]))) - (((np.tanh((np.tanh((data[:,6]))))) - (data[:,10]))))))/2.0)))/2.0)))) +
                0.250000*np.tanh((((((((((((data[:,10]) + (data[:,4]))) + (((data[:,10]) + (((data[:,0]) * (data[:,10]))))))/2.0)) + (((data[:,4]) * (((data[:,10]) + (data[:,4]))))))/2.0)) + (data[:,7]))/2.0)) +
                0.250000*np.tanh((((((data[:,11]) + (((((data[:,18]) / 2.0)) * (((data[:,11]) * (data[:,18]))))))/2.0)) * (((data[:,18]) * (((((((((data[:,18]) * (((((((data[:,11]) / 2.0)) / 2.0)) * (data[:,18]))))) / 2.0)) / 2.0)) * ((-1.0*((data[:,21])))))))))) +
                0.250000*np.tanh((((-1.0*((((((((((((data[:,2]) - (data[:,2]))) * (np.tanh((data[:,2]))))) + (np.tanh((((data[:,14]) * 2.0)))))/2.0)) + (data[:,2]))/2.0))))) - (data[:,2]))))
    
    def GP_class_5(self,data):
        return(0.250000*np.tanh(((((((((((3.46574378013610840)) + ((((np.tanh(((3.46574378013610840)))) + ((8.46705245971679688)))/2.0)))/2.0)) * 2.0)) + ((((3.46574378013610840)) + ((4.0)))))) + ((((4.0)) * 2.0)))) +
                0.250000*np.tanh((((10.55856513977050781)) / 2.0)) +
                0.250000*np.tanh((((3.76695251464843750)) / 2.0)) +
                0.250000*np.tanh((((((6.0)) / 2.0)) + ((((8.57984828948974609)) * 2.0)))) +
                0.250000*np.tanh((((((((((((3.42168045043945312)) + (data[:,7]))) + (np.tanh((data[:,7]))))) + (np.tanh((np.tanh((np.tanh(((3.42168045043945312)))))))))/2.0)) + (data[:,7]))/2.0)) +
                0.250000*np.tanh(((((((((((data[:,6]) - (data[:,9]))) + (data[:,7]))/2.0)) * 2.0)) + ((((((data[:,7]) + (data[:,9]))/2.0)) - (data[:,9]))))/2.0)) +
                0.250000*np.tanh(((((((((((data[:,22]) + (data[:,22]))) + (data[:,11]))/2.0)) + (data[:,22]))/2.0)) / 2.0)) +
                0.250000*np.tanh((((((((((0.76044219732284546)) / 2.0)) + ((0.76044219732284546)))/2.0)) + (np.tanh((np.tanh((((((((0.76043862104415894)) / 2.0)) + ((1.0)))/2.0)))))))/2.0)) +
                0.250000*np.tanh(((((((data[:,12]) + (np.tanh((data[:,9]))))/2.0)) + (((((((((((data[:,12]) + (((((0.0)) + (np.tanh((((((((data[:,7]) + (data[:,6]))/2.0)) + (data[:,7]))/2.0)))))/2.0)))/2.0)) / 2.0)) / 2.0)) + (data[:,7]))/2.0)))/2.0)) +
                0.250000*np.tanh(((((((((data[:,20]) / 2.0)) / 2.0)) * (((np.tanh((data[:,20]))) * (((data[:,20]) / 2.0)))))) * ((((((data[:,20]) / 2.0)) + (((data[:,20]) / 2.0)))/2.0)))))
    
    def GP_class_6(self,data):
        return(0.250000*np.tanh(((((((11.68076229095458984)) + (((((11.68075847625732422)) + ((((11.47270107269287109)) + (((((11.47269725799560547)) + ((11.68076229095458984)))/2.0)))))/2.0)))/2.0)) + ((((4.0)) * 2.0)))) +
                0.250000*np.tanh((((8.0)) + ((((10.0)) + ((6.59042119979858398)))))) +
                0.250000*np.tanh(((((((((((13.33760547637939453)) + (((((3.86388397216796875)) + (((((((8.18321037292480469)) + (data[:,4]))) + (((((((5.95386552810668945)) + ((12.93883609771728516)))) + (((((10.73907089233398438)) + ((4.0)))/2.0)))/2.0)))/2.0)))/2.0)))/2.0)) + (np.tanh(((3.0)))))) - ((2.0)))) + (((((10.61559963226318359)) + (data[:,1]))/2.0)))) +
                0.250000*np.tanh(((np.tanh(((((((((((7.13513946533203125)) + ((12.48778533935546875)))/2.0)) + (((((((7.13513565063476562)) - ((8.12031841278076172)))) + (((((((((7.13513565063476562)) + ((((10.69830417633056641)) + ((10.69830799102783203)))))) + ((10.69830799102783203)))) + ((7.83078956604003906)))/2.0)))/2.0)))/2.0)) * 2.0)))) + ((3.0)))) +
                0.250000*np.tanh(((((((((((data[:,7]) - (data[:,0]))) * 2.0)) - (data[:,7]))) + (data[:,7]))) + (data[:,0]))) +
                0.250000*np.tanh(((((data[:,7]) / 2.0)) / 2.0)) +
                0.250000*np.tanh((((((((((((((((1.0)) + (((((((data[:,2]) + (((((((1.0)) * (data[:,2]))) + (data[:,14]))/2.0)))/2.0)) + ((1.0)))/2.0)))) + ((((1.0)) - (((data[:,2]) * (data[:,14]))))))/2.0)) - (data[:,2]))) * 2.0)) + (data[:,14]))/2.0)) + (data[:,14]))) +
                0.250000*np.tanh(((np.tanh((np.tanh(((((((data[:,19]) * ((0.0)))) + ((0.0)))/2.0)))))) / 2.0)) +
                0.250000*np.tanh((((((((0.0)) / 2.0)) / 2.0)) * (((((((((data[:,15]) / 2.0)) * ((0.0)))) / 2.0)) * ((((-1.0*(((0.0))))) / 2.0)))))) +
                0.250000*np.tanh(((((data[:,15]) * (((((((((((((np.tanh(((((0.09032609313726425)) / 2.0)))) / 2.0)) / 2.0)) / 2.0)) / 2.0)) / 2.0)) / 2.0)))) / 2.0)))
    
    def GP_class_7(self,data):
        return(0.250000*np.tanh((10.27210903167724609)) +
                0.250000*np.tanh((((((((((10.0)) + (((((((((8.26972770690917969)) + ((11.06334972381591797)))/2.0)) * 2.0)) / 2.0)))) + ((((((5.0)) + ((((5.0)) + ((7.92727756500244141)))))) * 2.0)))) * ((7.92728137969970703)))) + ((10.0)))) +
                0.250000*np.tanh((8.42519950866699219)) +
                0.250000*np.tanh(((((((data[:,14]) + (data[:,14]))) + ((((((1.59392631053924561)) + (data[:,14]))) + (data[:,14]))))) + (np.tanh((((((data[:,14]) + (((((((3.0)) + (((data[:,14]) * 2.0)))/2.0)) + ((1.59391915798187256)))))) + ((1.59392631053924561)))))))) +
                0.250000*np.tanh((((data[:,7]) + (((data[:,0]) + (((((((data[:,7]) + (data[:,4]))/2.0)) + (((data[:,7]) * 2.0)))/2.0)))))/2.0)) +
                0.250000*np.tanh((((((((((((((data[:,9]) - (((data[:,0]) - (data[:,0]))))) - (((data[:,0]) / 2.0)))) / 2.0)) - (((data[:,21]) - (data[:,7]))))) + (data[:,21]))/2.0)) - (data[:,0]))) +
                0.250000*np.tanh((((((((((2.0)) + ((((((data[:,21]) + ((((data[:,13]) + (((((((((((data[:,13]) * 2.0)) + (data[:,21]))/2.0)) + (data[:,13]))/2.0)) * 2.0)))/2.0)))/2.0)) - (np.tanh(((((0.0)) - (data[:,21]))))))))) + ((1.0)))/2.0)) + (data[:,13]))/2.0)) +
                0.250000*np.tanh(((((data[:,14]) / 2.0)) / 2.0)) +
                0.250000*np.tanh((((((((((((((((-1.0*(((((((data[:,21]) / 2.0)) + (data[:,8]))/2.0))))) / 2.0)) / 2.0)) / 2.0)) + (data[:,9]))/2.0)) / 2.0)) + ((((((((((((data[:,7]) / 2.0)) / 2.0)) / 2.0)) / 2.0)) + (data[:,7]))/2.0)))/2.0)) +
                0.250000*np.tanh(((((((np.tanh(((1.0)))) / 2.0)) / 2.0)) / 2.0)))
    
    def GP_class_8(self,data):
        return(0.250000*np.tanh(((((((data[:,0]) + (((((6.71804094314575195)) + (data[:,15]))/2.0)))) + ((((((11.56385326385498047)) * 2.0)) * ((((10.48986148834228516)) + ((11.56385326385498047)))))))) / 2.0)) +
                0.250000*np.tanh((((((((((data[:,14]) / 2.0)) + ((2.83755254745483398)))/2.0)) + ((((((data[:,12]) + (data[:,14]))/2.0)) + (data[:,14]))))) + ((((data[:,9]) + (((data[:,12]) + (data[:,14]))))/2.0)))) +
                0.250000*np.tanh(((((((((data[:,6]) + (((((((3.50821948051452637)) + ((((data[:,21]) + ((11.92932796478271484)))/2.0)))) + ((1.0)))/2.0)))) + (data[:,10]))/2.0)) + (data[:,21]))/2.0)) +
                0.250000*np.tanh(((data[:,14]) + (np.tanh((np.tanh(((-1.0*((((data[:,22]) - (data[:,13])))))))))))) +
                0.250000*np.tanh(((((1.0)) + ((((data[:,7]) + (data[:,7]))/2.0)))/2.0)) +
                0.250000*np.tanh(((((((((data[:,21]) / 2.0)) / 2.0)) * (((((((np.tanh((((np.tanh((data[:,13]))) / 2.0)))) / 2.0)) + (((((((np.tanh(((((data[:,21]) + (data[:,21]))/2.0)))) / 2.0)) - (data[:,12]))) / 2.0)))) / 2.0)))) - (data[:,12]))) +
                0.250000*np.tanh((((data[:,7]) + (np.tanh(((((data[:,12]) + (((data[:,7]) * ((((data[:,13]) + (data[:,13]))/2.0)))))/2.0)))))/2.0)) +
                0.250000*np.tanh(((((((data[:,13]) / 2.0)) / 2.0)) / 2.0)) +
                0.250000*np.tanh((((((((((np.tanh((data[:,9]))) + (data[:,13]))/2.0)) + ((((((((data[:,21]) / 2.0)) * 2.0)) + ((((3.94983267784118652)) / 2.0)))/2.0)))/2.0)) + ((((np.tanh(((3.94983267784118652)))) + (data[:,9]))/2.0)))/2.0)) +
                0.250000*np.tanh(((((((((((((((data[:,14]) * (((((((((data[:,19]) / 2.0)) * (np.tanh((((data[:,2]) / 2.0)))))) / 2.0)) / 2.0)))) / 2.0)) / 2.0)) / 2.0)) / 2.0)) / 2.0)) * (((((data[:,11]) / 2.0)) / 2.0)))))
    
    def GP_class_9(self,data):
        return(0.250000*np.tanh(((((((data[:,14]) * 2.0)) / 2.0)) + (((((data[:,14]) + (np.tanh((((data[:,14]) * 2.0)))))) + (data[:,14]))))) +
                0.250000*np.tanh(((data[:,9]) - (((((-1.0*((data[:,11])))) + (data[:,9]))/2.0)))) +
                0.250000*np.tanh(((data[:,21]) * 2.0)) +
                0.250000*np.tanh((((((data[:,14]) + (data[:,14]))/2.0)) + ((((data[:,13]) + ((((np.tanh((((data[:,9]) + ((6.0)))))) + (data[:,9]))/2.0)))/2.0)))) +
                0.250000*np.tanh(np.tanh(((1.0)))) +
                0.250000*np.tanh(((np.tanh((np.tanh(((((np.tanh((((data[:,20]) * (((((((np.tanh((((data[:,20]) / 2.0)))) + (data[:,12]))/2.0)) + ((((data[:,15]) + ((((data[:,17]) + (data[:,9]))/2.0)))/2.0)))/2.0)))))) + (data[:,9]))/2.0)))))) / 2.0)) +
                0.250000*np.tanh(((((data[:,2]) - (data[:,11]))) / 2.0)) +
                0.250000*np.tanh((((((((data[:,13]) / 2.0)) + (((data[:,7]) + (data[:,7]))))/2.0)) - (((((data[:,22]) + (((((data[:,14]) * (((((data[:,7]) * (((data[:,13]) - ((((data[:,7]) + (((((data[:,13]) / 2.0)) + (data[:,7]))))/2.0)))))) / 2.0)))) * 2.0)))) / 2.0)))) +
                0.250000*np.tanh(((((((((((((((data[:,1]) / 2.0)) + ((1.0)))/2.0)) + ((1.0)))/2.0)) + ((((((((((1.0)) / 2.0)) + (np.tanh(((1.0)))))/2.0)) + (np.tanh((((((1.0)) + ((1.0)))/2.0)))))/2.0)))/2.0)) + ((((data[:,20]) + ((1.0)))/2.0)))/2.0)) +
                0.250000*np.tanh(((data[:,1]) * ((((data[:,14]) + (np.tanh(((((0.0)) / 2.0)))))/2.0)))))
    
    def GP_class_10(self,data):
        return(0.250000*np.tanh((((((((((data[:,2]) + (((data[:,9]) * (np.tanh((data[:,14]))))))/2.0)) + ((((((data[:,14]) + (data[:,20]))/2.0)) + (data[:,0]))))/2.0)) + (data[:,14]))/2.0)) +
                0.250000*np.tanh((((((data[:,9]) + ((-1.0*((((((((2.04309988021850586)) + (((data[:,11]) + (((np.tanh((data[:,0]))) / 2.0)))))/2.0)) * 2.0))))))/2.0)) + (data[:,2]))) +
                0.250000*np.tanh(data[:,21]) +
                0.250000*np.tanh((((((((data[:,14]) + (np.tanh((((data[:,6]) / 2.0)))))/2.0)) + (data[:,14]))) + (((((data[:,14]) * 2.0)) + ((((data[:,13]) + (data[:,14]))/2.0)))))) +
                0.250000*np.tanh(data[:,9]) +
                0.250000*np.tanh(((((((data[:,15]) * 2.0)) / 2.0)) / 2.0)) +
                0.250000*np.tanh((((((((2.72836518287658691)) * ((((1.0)) / 2.0)))) / 2.0)) / 2.0)) +
                0.250000*np.tanh(((((np.tanh((((data[:,7]) + (np.tanh(((((((((((0.0)) / 2.0)) / 2.0)) / 2.0)) / 2.0)))))))) / 2.0)) / 2.0)) +
                0.250000*np.tanh(np.tanh((((np.tanh(((11.43236827850341797)))) / 2.0)))) +
                0.250000*np.tanh(np.tanh((((data[:,7]) / 2.0)))))
    
    def GP_class_11(self,data):
        return(0.250000*np.tanh((-1.0*((((((((((((((data[:,5]) - (((data[:,9]) - ((((-1.0*(((11.40053558349609375))))) / 2.0)))))) + ((((11.40053558349609375)) + ((((11.40053558349609375)) * 2.0)))))) / 2.0)) - (data[:,7]))) - ((-1.0*(((11.40053939819335938))))))) * 2.0))))) +
                0.250000*np.tanh(((((data[:,14]) * 2.0)) + (data[:,14]))) +
                0.250000*np.tanh(((data[:,2]) + (((((data[:,5]) + (data[:,3]))) + ((((data[:,2]) + (((data[:,3]) * 2.0)))/2.0)))))) +
                0.250000*np.tanh(((((((((((data[:,21]) + (data[:,21]))) + ((-1.0*(((((np.tanh((np.tanh((((data[:,14]) / 2.0)))))) + (data[:,21]))/2.0))))))/2.0)) * 2.0)) + ((((data[:,14]) + (data[:,3]))/2.0)))/2.0)) +
                0.250000*np.tanh(((((((data[:,0]) + ((((data[:,9]) + ((((data[:,9]) + (((((((((((((((data[:,9]) / 2.0)) - (data[:,10]))) + (data[:,5]))/2.0)) - (data[:,9]))) + (data[:,9]))/2.0)) / 2.0)))/2.0)))/2.0)))/2.0)) + (data[:,9]))/2.0)) +
                0.250000*np.tanh((((((((data[:,13]) + ((((data[:,13]) + (data[:,13]))/2.0)))) + (data[:,1]))/2.0)) / 2.0)) +
                0.250000*np.tanh(data[:,14]) +
                0.250000*np.tanh(((((data[:,2]) / 2.0)) / 2.0)) +
                0.250000*np.tanh((((data[:,9]) + (data[:,9]))/2.0)) +
                0.250000*np.tanh(data[:,2]))
    
    def GP_class_12(self,data):
        return(0.250000*np.tanh((((((((-1.0*((data[:,18])))) - (np.tanh((data[:,9]))))) + ((8.0)))) - ((14.74865913391113281)))) +
                0.250000*np.tanh((((((data[:,21]) + (np.tanh((data[:,9]))))) + ((((((((data[:,5]) / 2.0)) * 2.0)) + (data[:,5]))/2.0)))/2.0)) +
                0.250000*np.tanh(((((data[:,14]) + (data[:,14]))) + (((data[:,14]) * 2.0)))) +
                0.250000*np.tanh(((data[:,2]) + (((((((data[:,14]) + ((((np.tanh((data[:,10]))) + (data[:,17]))/2.0)))/2.0)) + (data[:,21]))/2.0)))) +
                0.250000*np.tanh((((data[:,9]) + ((((((((((((data[:,9]) + ((((((data[:,9]) / 2.0)) + (((data[:,9]) * 2.0)))/2.0)))/2.0)) + ((-1.0*((data[:,21])))))/2.0)) * (data[:,9]))) + (((((np.tanh((((data[:,21]) / 2.0)))) / 2.0)) / 2.0)))/2.0)))/2.0)) +
                0.250000*np.tanh(((((0.0)) + (data[:,13]))/2.0)) +
                0.250000*np.tanh((((((((data[:,2]) / 2.0)) * 2.0)) + (data[:,7]))/2.0)) +
                0.250000*np.tanh((((((data[:,14]) * 2.0)) + (np.tanh((data[:,14]))))/2.0)) +
                0.250000*np.tanh(np.tanh((((((((((-1.0*((((np.tanh((data[:,9]))) / 2.0))))) / 2.0)) / 2.0)) + (((data[:,3]) * (data[:,13]))))/2.0)))) +
                0.250000*np.tanh((-1.0*((((((((-1.0*(((((((-1.0*((data[:,18])))) - ((((0.0)) / 2.0)))) / 2.0))))) + (np.tanh(((((-1.0*(((-1.0*((data[:,22]))))))) / 2.0)))))/2.0)) / 2.0))))))
    
    def GP_class_13(self,data):
        return(0.250000*np.tanh(((((np.tanh((np.tanh((data[:,2]))))) - ((10.0)))) - (np.tanh((((((data[:,7]) - ((((((6.0)) - ((4.87243366241455078)))) - ((9.0)))))) - ((14.80352973937988281)))))))) +
                0.250000*np.tanh(((np.tanh((((((3.0)) + (data[:,1]))/2.0)))) - ((7.0)))) +
                0.250000*np.tanh(((((np.tanh((np.tanh((np.tanh((data[:,6]))))))) * 2.0)) - ((6.10337877273559570)))) +
                0.250000*np.tanh(((((((data[:,9]) + (data[:,14]))) + (data[:,14]))) + (data[:,14]))) +
                0.250000*np.tanh(((((((np.tanh(((((-1.0*(((((((data[:,2]) + ((10.0)))/2.0)) / 2.0))))) - ((13.28130435943603516)))))) + ((10.0)))) - ((13.28130435943603516)))) + ((-1.0*((data[:,5])))))) +
                0.250000*np.tanh(((((((((((data[:,20]) + (((((((((data[:,20]) * 2.0)) * 2.0)) * 2.0)) + (data[:,13]))))/2.0)) + (((data[:,9]) + (((data[:,9]) + (((data[:,6]) + (data[:,9]))))))))/2.0)) + (data[:,20]))) + (data[:,15]))) +
                0.250000*np.tanh(((((((((data[:,14]) / 2.0)) + ((((data[:,9]) + (((data[:,15]) + (((data[:,5]) + ((((((data[:,9]) + (((data[:,5]) + (data[:,14]))))) + (data[:,9]))/2.0)))))))/2.0)))) * 2.0)) + (data[:,14]))) +
                0.250000*np.tanh((((data[:,13]) + (data[:,7]))/2.0)) +
                0.250000*np.tanh(((data[:,9]) + ((((data[:,3]) + ((((data[:,21]) + (data[:,3]))/2.0)))/2.0)))) +
                0.250000*np.tanh((((data[:,2]) + (((((((((((((((data[:,20]) + (data[:,1]))) + (((data[:,2]) + (data[:,14]))))) + (data[:,2]))) + (np.tanh((data[:,2]))))/2.0)) + (((data[:,14]) + (data[:,2]))))) + (data[:,14]))/2.0)))/2.0)))
    
    def GP_class_14(self,data):
        return(0.250000*np.tanh((((((((((((5.54024744033813477)) - ((((((9.0)) * 2.0)) * 2.0)))) - (data[:,0]))) - ((((7.0)) + ((4.74105215072631836)))))) - (((((7.0)) + (((data[:,5]) * 2.0)))/2.0)))) - ((9.0)))) +
                0.250000*np.tanh(((((((((9.0)) - ((14.27298927307128906)))) - (data[:,0]))) + (np.tanh(((((12.77708148956298828)) - ((14.80560111999511719)))))))/2.0)) +
                0.250000*np.tanh(((((((0.0)) + ((-1.0*((((data[:,22]) + (((((((((13.30077362060546875)) - (np.tanh((data[:,4]))))) + (((data[:,22]) / 2.0)))/2.0)) / 2.0))))))))/2.0)) - (((np.tanh((((data[:,7]) - (data[:,7]))))) / 2.0)))) +
                0.250000*np.tanh(((((((data[:,6]) + (data[:,14]))/2.0)) + (((np.tanh((((((data[:,14]) + (data[:,14]))) + (((data[:,6]) - (((data[:,5]) + (((np.tanh((data[:,21]))) + ((((data[:,14]) + ((((data[:,14]) + (data[:,14]))/2.0)))/2.0)))))))))))) + (data[:,5]))))/2.0)) +
                0.250000*np.tanh((((((((((((((((((data[:,2]) + (data[:,16]))) + ((((((data[:,2]) / 2.0)) + (data[:,9]))/2.0)))/2.0)) + (np.tanh((data[:,10]))))/2.0)) + (data[:,9]))) + (data[:,9]))/2.0)) + (data[:,21]))) + (data[:,3]))) +
                0.250000*np.tanh((((data[:,13]) + (((((((((((data[:,3]) / 2.0)) / 2.0)) / 2.0)) * (data[:,3]))) / 2.0)))/2.0)) +
                0.250000*np.tanh(((data[:,14]) + (((((((data[:,14]) / 2.0)) + (data[:,14]))) * 2.0)))) +
                0.250000*np.tanh(data[:,9]) +
                0.250000*np.tanh(((data[:,1]) / 2.0)) +
                0.250000*np.tanh((((((data[:,2]) + ((((((data[:,13]) + (data[:,14]))) + (data[:,13]))/2.0)))) + (np.tanh((data[:,2]))))/2.0)))
    
    def GP_class_15(self,data):
        return(0.250000*np.tanh((((((((4.0)) + ((((8.0)) / 2.0)))) - ((9.56193733215332031)))) - (((((np.tanh(((13.93556308746337891)))) - (data[:,18]))) + ((13.93556308746337891)))))) +
                0.250000*np.tanh(((data[:,4]) - ((((12.76094818115234375)) - (data[:,8]))))) +
                0.250000*np.tanh((((-1.0*(((12.35446166992187500))))) - (((((12.35446166992187500)) + ((((((12.35446166992187500)) - ((-1.0*(((0.0))))))) - (data[:,0]))))/2.0)))) +
                0.250000*np.tanh(((data[:,21]) - ((14.46367263793945312)))) +
                0.250000*np.tanh(((((data[:,2]) + ((((data[:,9]) + ((-1.0*(((-1.0*(((-1.0*((((data[:,7]) * (((data[:,21]) * 2.0))))))))))))))/2.0)))) + (data[:,9]))) +
                0.250000*np.tanh(((((((((((data[:,7]) + (data[:,3]))/2.0)) + (data[:,20]))/2.0)) - ((((2.33354020118713379)) / 2.0)))) - ((((2.0)) + ((8.78930377960205078)))))) +
                0.250000*np.tanh(((((((data[:,14]) + (((((((data[:,14]) + (((data[:,15]) + (data[:,14]))))) + (((data[:,15]) + (data[:,14]))))) + (((data[:,14]) / 2.0)))))) + (data[:,14]))) + (data[:,14]))) +
                0.250000*np.tanh(((((((((data[:,13]) + (data[:,13]))) + (((data[:,13]) + (data[:,12]))))) + (data[:,9]))) + ((((((data[:,9]) + (((data[:,13]) + ((((data[:,9]) + (((((data[:,13]) * 2.0)) / 2.0)))/2.0)))))/2.0)) * 2.0)))) +
                0.250000*np.tanh((((data[:,9]) + ((((((data[:,9]) + (data[:,9]))/2.0)) / 2.0)))/2.0)) +
                0.250000*np.tanh((((((data[:,14]) + (data[:,8]))) + (data[:,14]))/2.0)))
    
    def GP_class_16(self,data):
        return(0.250000*np.tanh(((((((((((((9.37592792510986328)) - ((13.36316204071044922)))) + ((11.89122962951660156)))/2.0)) - ((((9.0)) - (((((((data[:,13]) / 2.0)) / 2.0)) + ((((-1.0*((((data[:,4]) / 2.0))))) * 2.0)))))))) - ((12.24639320373535156)))) - ((11.89122581481933594)))) +
                0.250000*np.tanh(((((np.tanh(((4.51821422576904297)))) - ((13.23712635040283203)))) - ((((((13.23712635040283203)) + (((data[:,22]) - ((((11.40539550781250000)) - (((((9.0)) + ((((((((data[:,7]) + (((data[:,17]) / 2.0)))/2.0)) * 2.0)) / 2.0)))/2.0)))))))) - (data[:,5]))))) +
                0.250000*np.tanh(((data[:,14]) - ((((4.0)) + (((((13.41770648956298828)) + ((12.42538261413574219)))/2.0)))))) +
                0.250000*np.tanh((((5.0)) - ((9.27778816223144531)))) +
                0.250000*np.tanh(((((data[:,16]) - (((((((data[:,1]) - (((np.tanh((((((((9.29507255554199219)) - (((((((14.34461498260498047)) * 2.0)) + (((((7.0)) + ((9.10683441162109375)))/2.0)))/2.0)))) + ((4.21145153045654297)))/2.0)))) * 2.0)))) - ((9.10683441162109375)))) * 2.0)))) - ((((14.34461498260498047)) * 2.0)))) +
                0.250000*np.tanh((((((((((data[:,17]) / 2.0)) + (((data[:,13]) + (data[:,9]))))/2.0)) + (((data[:,17]) + (data[:,9]))))) + (data[:,0]))) +
                0.250000*np.tanh(((data[:,13]) + (((data[:,14]) - (data[:,21]))))) +
                0.250000*np.tanh((((((data[:,14]) + (data[:,5]))/2.0)) + (((((((data[:,9]) / 2.0)) + (data[:,14]))) / 2.0)))) +
                0.250000*np.tanh((((((data[:,17]) + (data[:,9]))) + (((((data[:,17]) / 2.0)) + ((((((((((data[:,9]) + (data[:,9]))) - ((3.0)))) + (data[:,7]))) + (data[:,9]))/2.0)))))/2.0)) +
                0.250000*np.tanh(np.tanh(((((((((((((((data[:,2]) + (((((((((((data[:,22]) + (data[:,21]))/2.0)) + (data[:,2]))/2.0)) * 2.0)) + ((((((data[:,21]) * 2.0)) + (data[:,21]))/2.0)))))) + (data[:,2]))/2.0)) + (data[:,17]))/2.0)) + (np.tanh((data[:,8]))))/2.0)) + (data[:,2]))))))
    
    def GP_class_17(self,data):
        return(0.250000*np.tanh(((((data[:,14]) - ((((data[:,5]) + (((((14.43326377868652344)) + ((6.0)))/2.0)))/2.0)))) - (((((((10.0)) + ((14.84462165832519531)))/2.0)) - ((((7.0)) + (data[:,13]))))))) +
                0.250000*np.tanh((((((((((data[:,14]) + (((((((((data[:,11]) - ((9.71331787109375000)))) / 2.0)) + ((9.0)))) - ((3.0)))))/2.0)) - ((10.0)))) - ((((10.0)) + ((((9.0)) - ((((((10.0)) - ((9.0)))) / 2.0)))))))) - ((10.0)))) +
                0.250000*np.tanh((((((((((8.0)) - ((12.78277492523193359)))) - (data[:,10]))) - (np.tanh(((((((7.0)) - ((14.95321178436279297)))) - ((8.0)))))))) - ((14.95321178436279297)))) +
                0.250000*np.tanh((((7.45682907104492188)) - ((14.98023796081542969)))) +
                0.250000*np.tanh(((((((((((7.0)) + (((((np.tanh((((((((8.0)) * (data[:,8]))) + (data[:,11]))/2.0)))) / 2.0)) - ((8.0)))))/2.0)) - (data[:,20]))) - ((((8.0)) * 2.0)))) + ((8.0)))) +
                0.250000*np.tanh(((data[:,8]) + (((data[:,4]) + (((np.tanh((data[:,4]))) + (((data[:,13]) + (data[:,13]))))))))) +
                0.250000*np.tanh(data[:,14]) +
                0.250000*np.tanh(((((((((((data[:,14]) + (np.tanh((((data[:,3]) * (data[:,13]))))))) + (data[:,14]))) + (data[:,0]))) + (data[:,8]))) / 2.0)) +
                0.250000*np.tanh((((((data[:,9]) + (np.tanh(((((((data[:,19]) * (((((((data[:,1]) + (((((((data[:,21]) + (data[:,9]))/2.0)) + (data[:,6]))/2.0)))/2.0)) + (data[:,17]))/2.0)))) + (((data[:,15]) / 2.0)))/2.0)))))/2.0)) * 2.0)) +
                0.250000*np.tanh(((data[:,1]) + (((((data[:,3]) + (data[:,15]))) + (np.tanh(((((data[:,20]) + (((data[:,1]) * 2.0)))/2.0)))))))))
    
    def GP_class_18(self,data):
        return(0.250000*np.tanh((((((((((12.59485149383544922)) - ((12.59485149383544922)))) - ((12.59485149383544922)))) - ((((11.47756862640380859)) / 2.0)))) - ((12.59485149383544922)))) +
                0.250000*np.tanh((((((-1.0*(((11.26121807098388672))))) - (((((((((11.33140659332275391)) - ((-1.0*(((9.76293182373046875))))))) - (((((11.33140659332275391)) + ((8.0)))/2.0)))) + ((9.76293182373046875)))/2.0)))) - (data[:,11]))) +
                0.250000*np.tanh(((((((((((3.40250444412231445)) - (np.tanh(((14.86789989471435547)))))) - ((14.86789989471435547)))) + ((((((data[:,5]) + ((-1.0*(((((14.86789989471435547)) * 2.0))))))/2.0)) - ((14.86789989471435547)))))/2.0)) - (((((14.86789989471435547)) + (((((7.0)) + ((-1.0*((data[:,16])))))/2.0)))/2.0)))) +
                0.250000*np.tanh(((data[:,18]) - ((11.84332561492919922)))) +
                0.250000*np.tanh(((((np.tanh(((((8.96548557281494141)) * (data[:,12]))))) - ((-1.0*((((data[:,2]) * (((((data[:,4]) - ((-1.0*((data[:,9])))))) - ((9.0))))))))))) - ((8.96548557281494141)))) +
                0.250000*np.tanh(((((((10.0)) - (data[:,12]))) + (((((((((data[:,13]) - ((12.43707370758056641)))) - ((((12.43707370758056641)) - (((data[:,12]) / 2.0)))))) - ((((12.43707370758056641)) - ((((data[:,13]) + ((12.43707370758056641)))/2.0)))))) - ((12.43707370758056641)))))/2.0)) +
                0.250000*np.tanh(((((((np.tanh((((np.tanh((data[:,14]))) + (data[:,13]))))) * 2.0)) + (data[:,14]))) + (data[:,13]))) +
                0.250000*np.tanh(((np.tanh((data[:,14]))) + (data[:,14]))) +
                0.250000*np.tanh(((((data[:,9]) + (((((data[:,13]) + (data[:,4]))) + (data[:,13]))))) / 2.0)) +
                0.250000*np.tanh(((data[:,15]) - ((((data[:,3]) + ((5.0)))/2.0)))))
    
    def GP_class_19(self,data):
        return(0.250000*np.tanh((((((data[:,13]) + (data[:,13]))/2.0)) + ((((((data[:,8]) + ((((data[:,14]) + (data[:,14]))/2.0)))/2.0)) * 2.0)))) +
                0.250000*np.tanh((((data[:,3]) + (((data[:,8]) + (np.tanh(((((((-1.0*((((((9.05535793304443359)) + ((((data[:,14]) + (((((((data[:,1]) - (((data[:,12]) * 2.0)))) * ((-1.0*((data[:,17])))))) / 2.0)))/2.0)))/2.0))))) + (data[:,2]))) * 2.0)))))))/2.0)) +
                0.250000*np.tanh(((((((((data[:,3]) + (((((data[:,14]) + (data[:,14]))) * 2.0)))) + ((((data[:,14]) + (np.tanh((((data[:,14]) + (np.tanh((data[:,14]))))))))/2.0)))) * 2.0)) + ((((data[:,14]) + (data[:,2]))/2.0)))) +
                0.250000*np.tanh(((((data[:,8]) + (((data[:,4]) + (((data[:,13]) + (data[:,10]))))))) + (((((data[:,13]) + (((((data[:,13]) / 2.0)) + (data[:,9]))))) + (np.tanh(((((data[:,8]) + (data[:,8]))/2.0)))))))) +
                0.250000*np.tanh((((((data[:,19]) + (((((data[:,10]) + ((((data[:,11]) + (data[:,19]))/2.0)))) + (data[:,10]))))/2.0)) + (data[:,2]))) +
                0.250000*np.tanh((((((data[:,0]) + (((((((((data[:,0]) + (((((data[:,0]) - (data[:,11]))) / 2.0)))/2.0)) * 2.0)) + ((((((np.tanh((data[:,13]))) + (((data[:,0]) / 2.0)))/2.0)) / 2.0)))/2.0)))/2.0)) * 2.0)) +
                0.250000*np.tanh(((((-1.0*((data[:,15])))) + (((data[:,14]) + (data[:,14]))))/2.0)) +
                0.250000*np.tanh(((((((((data[:,0]) + (data[:,15]))/2.0)) + (((data[:,15]) * (data[:,15]))))/2.0)) - (data[:,11]))) +
                0.250000*np.tanh(data[:,13]) +
                0.250000*np.tanh(((((((((data[:,18]) + ((((data[:,4]) + (data[:,18]))/2.0)))) / 2.0)) + ((((data[:,18]) + (data[:,4]))/2.0)))) * (data[:,4])))    )

# Time for the actual submission

In [17]:
import warnings
warnings.filterwarnings("ignore")
from kaggle.competitions import nflrush

preds = []
env = nflrush.make_env()
iter_test = env.iter_test()
gp = GP()

for (test_df, sample_prediction_df) in tqdm_notebook(iter_test):
    basetable = create_features(test_df, deploy=True)
    basetable.drop(['GameId','PlayId'], axis=1, inplace=True)
    
    scaled_basetable = scaler.transform(basetable)
    
    p = [m.predict(scaled_basetable) for m in models]
    y_pred_nn = (np.array(p)).mean(0)
    #np.expm1(np.log1p(np.array(p)).mean(0))

    y_pred_gp = np.zeros((test_df.shape[0],199))
    ans = gp.GrabPredictions(scaled_basetable)
    y_pred_gp[:,96:96+20] = ans
    
    y_pred = (.6*y_pred_nn+.4*y_pred_gp)
    y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1).tolist()[0]

    preds_df = pd.DataFrame(data=[y_pred], columns=sample_prediction_df.columns)
    preds.append(preds_df)
    env.predict(preds_df)

env.write_submission_file()

ModuleNotFoundError: No module named 'kaggle'

In [ ]:
preds = np.vstack(preds)
## check whether prediction is submittable
print(np.mean(np.diff(preds, axis = 1) >= 0) == 1.0)
print(np.mean(preds > 1) == 0)

In [ ]:
print(losses)
print(np.mean(losses))
    